In [1]:
import pandas as pd
import numpy as np
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import string
import random
import string
from sklearn import linear_model
from implicit import bpr
import csv
import json
from random import sample 

In [2]:
def make_json(file):   
    data = {}
    with open(file, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)         
        for rows in csvReader:
            key = rows['id']
            data[key] = rows
    return data
def make_json_u(file):   
    data = {}
    with open(file, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)         
        for rows in csvReader:
            key = rows['u']
            data[key] = rows
    return data
def make_json_i(file):   
    data = []
    with open(file, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)         
        for rows in csvReader:
            data.append((rows['user_id'], rows['recipe_id'], rows))
    return data

In [3]:
all_interactions = make_json_i('Raw_interactions.csv')
data_train = make_json_i('interactions_train.csv')
data_val = make_json_i('interactions_validation.csv')
data_test = make_json_i('interactions_test.csv')
all_interactions[0]

('38094',
 '40893',
 {'user_id': '38094',
  'recipe_id': '40893',
  'date': '2003-02-17',
  'rating': '4',
  'review': 'Great with a salad. Cooked on top of stove for 15 minutes.Added a shake of cayenne and a pinch of salt.  Used low fat sour cream.  Thanks.'})

In [4]:
userID, itemID = {},{}
flippedID = {}
for user, recipe, d in all_interactions:
    if not user in userID: userID[user] = len(userID)
    if not recipe in itemID: 
        itemID[recipe] = len(itemID)
        flippedID[len(itemID)] = recipe
nUsers,nItems = len(userID),len(itemID)
nItems

231637

In [5]:
Xui = scipy.sparse.lil_matrix((nUsers, nItems))
for u,r,d in all_interactions:
    Xui[userID[u],itemID[r]] = 1
    
Xui_csr = scipy.sparse.csr_matrix(Xui)

In [6]:
model = bpr.BayesianPersonalizedRanking(factors = 5)
model.fit(Xui_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

In [7]:
predictions = []
for u, i, d in data_test:
    comp = model.recommend(userID[u], Xui_csr[userID[u]], N = 5000)
    if itemID[i] in comp[0]:
        predictions.append([u, i, 1])
    else:
        predictions.append([u, i, 0])
predictions

[['8937', '44551', 0],
 ['56680', '126118', 0],
 ['349752', '219596', 0],
 ['628951', '82783', 0],
 ['92816', '435013', 0],
 ['280271', '228179', 0],
 ['345569', '186470', 0],
 ['724516', '298748', 0],
 ['176615', '118119', 0],
 ['56112', '166712', 0],
 ['537179', '78641', 0],
 ['222478', '437144', 0],
 ['22898', '65976', 0],
 ['857489', '311630', 0],
 ['1056692', '312579', 0],
 ['89831', '74778', 0],
 ['107583', '141156', 0],
 ['461834', '479953', 0],
 ['148715', '99695', 0],
 ['1206102', '207099', 0],
 ['213139', '366693', 0],
 ['128541', '50194', 0],
 ['865936', '221247', 0],
 ['592157', '80001', 0],
 ['323186', '482126', 0],
 ['585496', '173241', 0],
 ['320449', '282862', 0],
 ['220348', '394419', 0],
 ['2324285', '484976', 0],
 ['459426', '223667', 0],
 ['1248992', '223667', 0],
 ['41578', '214340', 0],
 ['583349', '301423', 0],
 ['1363685', '200334', 0],
 ['6357', '51255', 0],
 ['650039', '270812', 0],
 ['326176', '193693', 0],
 ['66983', '99932', 0],
 ['65056', '42144', 0],
 ['4

In [11]:
# predict whether a user would rate a certain item, accuracy is going to be super low.... but thats ok

In [18]:
#baseline

recipeCount = defaultdict(int)
totalMade = 0

for user,recipe,_ in data_train:
    recipeCount[recipe] += 1
    totalMade += 1

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalMade/1.1: break

predictions = []
for u,i,d in data_test:
    if i in return1:
        predictions.append([u, i, 1])
    else:
        predictions.append([u, i, 0])

predictions

[['8937', '44551', 0],
 ['56680', '126118', 0],
 ['349752', '219596', 0],
 ['628951', '82783', 0],
 ['92816', '435013', 0],
 ['280271', '228179', 0],
 ['345569', '186470', 0],
 ['724516', '298748', 0],
 ['176615', '118119', 0],
 ['56112', '166712', 0],
 ['537179', '78641', 0],
 ['222478', '437144', 0],
 ['22898', '65976', 0],
 ['857489', '311630', 0],
 ['1056692', '312579', 0],
 ['89831', '74778', 0],
 ['107583', '141156', 0],
 ['461834', '479953', 0],
 ['148715', '99695', 0],
 ['1206102', '207099', 0],
 ['213139', '366693', 0],
 ['128541', '50194', 0],
 ['865936', '221247', 0],
 ['592157', '80001', 0],
 ['323186', '482126', 0],
 ['585496', '173241', 0],
 ['320449', '282862', 0],
 ['220348', '394419', 0],
 ['2324285', '484976', 0],
 ['459426', '223667', 0],
 ['1248992', '223667', 0],
 ['41578', '214340', 0],
 ['583349', '301423', 0],
 ['1363685', '200334', 0],
 ['6357', '51255', 0],
 ['650039', '270812', 0],
 ['326176', '193693', 0],
 ['66983', '99932', 0],
 ['65056', '42144', 0],
 ['4